# **🧵 1: What is Threading?**

### **🔍 Definition:**

**Threading** allows a program to run multiple parts (tasks/functions) *seemingly at the same time*, **within the same process**.

* Each **thread** runs independently.
* Useful when your code waits for I/O (like file read, network, API calls).
* **Python’s threading is limited by the GIL**, but still helps in many cases.

---

### 💡 Real-World Analogy:

Imagine you're cooking two dishes. You:

* Start boiling rice (takes time)
* While rice is cooking, you chop vegetables

You’re not “duplicated,” but you're **switching tasks** — like threads.

---

### 🧠 Key Terms:

| Term                              | Meaning                                                                          |
| --------------------------------- | -------------------------------------------------------------------------------- |
| **Thread**                        | Smallest unit of execution in a program                                          |
| **Multithreading**                | Running multiple threads (functions/tasks) concurrently                          |
| **GIL (Global Interpreter Lock)** | A CPython mechanism that allows only one thread to run Python bytecode at a time |


In [11]:
### 🧪 Simple Code Example:
import threading

def greet():
    print("Hello from a thread!")

# Create a thread
t = threading.Thread(target=greet)


t.start()
# Main thread continues
print("Hello from main thread")
# Start the thread

# Wait for the thread to finish
t.join()


Hello from a thread!
Hello from main thread



> Threads run concurrently — so order isn't guaranteed.

---

### ✅ What we just did:

* Created a thread with `target=greet`
* Started it with `start()`
* Waited for it with `join()` (optional, but recommended)


So A **thread** is the smallest unit of execution within a process. Python’s `threading` module allows us to run multiple threads **in parallel**, making it useful for **I/O-bound tasks**.

---

### 🆚 Threading vs Asyncio (Quick Recap)

| Feature          | `threading`                      | `asyncio`                      |
| ---------------- | -------------------------------- | ------------------------------ |
| Runs in          | Multiple OS threads              | One thread                     |
| Switching        | OS decides                       | Developer-controlled (`await`) |
| Good for         | Blocking I/O, GUI apps           | Async I/O (e.g., APIs)         |
| True parallelism | ❌ Not for CPU-bound (due to GIL) | ❌ Also no true parallelism     |

---

### ✅ Example: Without Threads

```python
import time

def greet(name):
    time.sleep(2)
    print(f"Hello, {name}!")

def main():
    greet("Alice")
    greet("Bob")

main()
```

**Takes \~4 seconds total** — because it runs one after another.

---

### ✅ Now With Threads

```python
import threading
import time

def greet(name):
    time.sleep(2)
    print(f"Hello, {name}!")

t1 = threading.Thread(target=greet, args=("Alice",))
t2 = threading.Thread(target=greet, args=("Bob",))

t1.start()
t2.start()

t1.join()
t2.join()

print("All done!")
```

**Takes \~2 seconds total** — because both greetings run in parallel on separate threads.


### **What Happens If We Don't join() a Thread?**

Absolutely! Here's a clear summary:

---

## ✅ Summary: What Happens If We Don't `join()` a Thread?

| Scenario                           | Behavior                                                          |
| ---------------------------------- | ----------------------------------------------------------------- |
| **Thread is non-daemon** (default) | Python waits for it before exiting, even without `join()`         |
| **Thread is daemon**               | Python terminates it as soon as the main program ends             |
| **You don’t use `join()`**         | The main program continues without waiting for the thread, but... |
|                                    | ...non-daemon threads will still run in background                |
|                                    | ...daemon threads will be killed **immediately** if still running |

---

### 🔁 Rule of Thumb:

* `join()` ensures the main thread **waits** for your thread to finish.
* Without `join()`:

  * ✅ Non-daemon thread? It probably finishes anyway.
  * ❌ Daemon thread? Might be **killed abruptly** before finishing.



In [18]:
#Normal
import threading
import time

def counter():
    for i in range(5):
        print(i)
        time.sleep(1)


t1 = threading.Thread(target = counter) # Non-Daemon Thread

t1.start()
print("main starts")
t1.join()
print("main Ends")

0main starts

1
2
3
4
main Ends


In [21]:
# Without Join, Non-daemon task
import threading
import time

def counter():
    for i in range(5):
        print(i)
        time.sleep(1)


t1 = threading.Thread(target = counter) # Non-Daemon Thread

t1.start()
print("main starts")
#t1.join()
print("main Ends")

0main starts
main Ends



IN colab, this works differently, but above code should finish counter function execution.
Ideal Output:
```
0
main starts
main Ends
1
2
3
4

```




In [20]:
# Without Join, daemon task
import threading
import time

def counter():
    for i in range(5):
        print(i)
        time.sleep(1)


t1 = threading.Thread(target = counter,daemon=True) # Non-Daemon Thread

t1.start()
print("main starts")
#t1.join()
print("main Ends")

0main starts
main Ends



# **✅ 2: Daemon Threads – Meaning and Behavior**

### 🧠 What is a Daemon Thread?

A **daemon thread** runs in the background and **doesn't block** the main program from exiting.

If all non-daemon threads are done, the Python program **will exit**, even if daemon threads are still running.

---

### ❗ Real-life Analogy:

A **daemon** thread is like a helper — if the boss (main thread) leaves, the helper is dismissed immediately without finishing.

---

### 🔧 Setting a Thread as Daemon:

```python
t = threading.Thread(target=some_func, daemon=True)
```

Or after creation:

```python
t = threading.Thread(target=some_func)
t.daemon = True
```

---

### 🧪 Example:

```python
import threading
import time

def long_task():
    for i in range(10):
        print(f"Working... {i}")
        time.sleep(1)

t = threading.Thread(target=long_task, daemon=True)
t.start()

print("Main thread finished!")
```

### 🧨 Output (in a script):

You might see only `Working... 0` or `1` and then it stops — because the **main program exits immediately**, and daemon thread is killed.

---

### 🔄 Daemon vs Non-Daemon Summary

| Feature             | Non-Daemon (Default) | Daemon                     |
| ------------------- | -------------------- | -------------------------- |
| Waits after main    | ✅ Yes                | ❌ No                       |
| Keeps process alive | ✅ Yes                | ❌ No                       |
| Use case            | Essential work       | Background/logging helpers |

---


# **✅  3: Thread Lifecycle – start(), is_alive(), and join()**

This step is all about **how threads behave over time**, and how we can check and manage them.

---

### 🔁 Common Thread Methods

| Method       | Description                                                |
| ------------ | ---------------------------------------------------------- |
| `start()`    | Begins the thread’s execution (runs the `target` function) |
| `is_alive()` | Returns `True` if the thread is still running              |
| `join()`     | Waits for the thread to finish                             |

---

### 🔧 Lifecycle Flow

1. **Create** the thread: `t = Thread(...)`
2. **Start** the thread: `t.start()`
3. **Check** if it’s alive: `t.is_alive()`
4. **Join** the thread: `t.join()` to wait for it

---

### 🧪 Example:

```python
import threading
import time

def worker():
    print("Worker starting...")
    time.sleep(3)
    print("Worker done!")

t = threading.Thread(target=worker)

print("Before starting:", t.is_alive())  # False
t.start()
print("After starting:", t.is_alive())   # True (thread is running)

t.join()
print("After join:", t.is_alive())       # False (thread finished)
```

---

### 🔍 Output will be something like:

```
Before starting: False
Worker starting...
After starting: True
Worker done!
After join: False
```


# **✅  4: Passing Arguments to Threads**

By default, threads just call the function you assign. But often, you'll want to **pass inputs** to that function — like names, durations, IDs, etc.

---

### 🧠 How to Pass Arguments to a Thread

Use the `args` parameter of `Thread()` to pass a tuple of arguments to the function.

```python
threading.Thread(target=function_name, args=(arg1, arg2, ...))
```

---

### 🔧 Example: Greet with a delay

```python
import threading
import time

def greet(name, delay):
    time.sleep(delay)
    print(f"Hello {name}, after {delay} seconds!")

t1 = threading.Thread(target=greet, args=("Alice", 2))
t2 = threading.Thread(target=greet, args=("Bob", 1))

t1.start()
t2.start()

t1.join()
t2.join()
```

---

### 📈 Output (order may vary):

```
Hello Bob, after 1 seconds!
Hello Alice, after 2 seconds!
```

Because both threads run in parallel — Bob has a shorter wait.


# **✅ 5: Running Multiple Threads and Managing Execution**

Sometimes, you’ll want to launch **many threads** (like 5, 10, or 100) — for tasks like:

* Downloading multiple files
* Making multiple API calls
* Processing files in parallel

Let’s see how to manage multiple threads with a loop.

---

### 🔧 Creating Threads in a Loop

You can store them in a list and start/join them one by one.

```python
import threading
import time

def worker(id):
    print(f"🧵 Thread {id} starting")
    time.sleep(2)
    print(f"✅ Thread {id} finished")

threads = []

for i in range(5):  # Launch 5 threads
    t = threading.Thread(target=worker, args=(i,))
    threads.append(t)
    t.start()

for t in threads:
    t.join()

print("🎉 All threads completed!")
```

---

### 🔍 How it works:

* Threads are stored in a list.
* First loop starts all threads.
* Second loop waits (`join`) for each one to finish.
* Total time ≈ 2 seconds (not 10), because they run in parallel.



# **✅  6: Race Conditions – What They Are and Why They’re Dangerous**

Great — now stepping into the **critical part** of multithreading.

---



### 🧠 What is a Race Condition?

A **race condition** occurs when:

* Two or more threads **access and modify shared data**
* At the **same time**
* Without proper control

The result is **unpredictable** behavior and **bugs that are hard to reproduce**.

---

### 🔧 Real-life Analogy

Imagine two people trying to update the same Google Sheet **at the same time** without coordination. One update may **overwrite** the other — that's a race condition.

---

### 🧪 Problem Example:

```python
import threading

counter = 0

def increment():
    global counter
    for _ in range(100000):
        counter += 1

threads = []
for _ in range(5):
    t = threading.Thread(target=increment)
    threads.append(t)
    t.start()

for t in threads:
    t.join()

print("Final counter value:", counter)
```

---

### ⚠️ Expected Output:

`500000` (100000 × 5)

### ❌ Actual Output:

Something **less than 500000** (varies every run)

### 📉 Why?

Because `counter += 1` is **not atomic** — it breaks into:

1. Read value
2. Add 1
3. Write back

If two threads read the same value at once, both may overwrite each other’s updates.



# **✅ 7: Thread Synchronization Using `Lock`**

Perfect! Let's now learn how to fix the race condition using synchronization.

### 🧠 What is a Lock?

A **Lock** ensures that **only one thread** can access a block of code (called the *critical section*) at a time. It’s like putting a “Do Not Disturb” sign on shared data.

---

### 🔧 Using `threading.Lock`

```python
lock = threading.Lock()

with lock:
    # critical section
    shared_data += 1
```

Or manually:

```python
lock.acquire()
try:
    # critical section
finally:
    lock.release()
```

---

### 🧪 Fixing Race Condition

```python
import threading

counter = 0
lock = threading.Lock()

def increment():
    global counter
    for _ in range(100000):
        with lock:  # lock this section
            counter += 1

threads = []

for _ in range(5):
    t = threading.Thread(target=increment)
    threads.append(t)
    t.start()

for t in threads:
    t.join()

print("Final counter value:", counter)
```

---

### ✅ Now the output will always be:

```
Final counter value: 500000
```

Because each thread must **wait its turn** to update `counter`.

---

### 🧠 Key Points:

| Term             | Meaning                                     |
| ---------------- | ------------------------------------------- |
| Lock             | Allows only 1 thread at a time in a section |
| Critical Section | Code that accesses shared data              |
| with lock        | Acquires and releases lock safely           |

---

# **🧠  GIL   🧠**

## ✅ What is GIL?

* **GIL** stands for **Global Interpreter Lock**.
* It is a **mutex (mutual exclusion lock)** in **CPython** (the default Python interpreter).
* Ensures that **only one thread** runs **Python bytecode** at a time.
* Exists to **protect internal memory management** and avoid race conditions in the interpreter itself.

---

## ✅ Why does Python have a GIL?

* CPython’s memory management (reference counting) is **not thread-safe**.
* GIL simplifies implementation by **avoiding complex locks** around internal objects.
* But this comes at the cost of **true parallelism** for CPU-bound tasks.

---

## ✅ When does GIL matter?

| Task Type           | Affected by GIL?                    | Notes                        |
| ------------------- | ----------------------------------- | ---------------------------- |
| **CPU-bound**       | ❌ Yes – threads are blocked         | Only 1 thread runs at a time |
| **I/O-bound**       | ✅ No – GIL is released during I/O   | Threads perform well         |
| **Multiprocessing** | ✅ No – each process has its own GIL | True parallelism             |

---

## ✅ GIL vs `threading.Lock`

| Feature            | GIL                           | `threading.Lock`                     |
| ------------------ | ----------------------------- | ------------------------------------ |
| Who controls it?   | Python interpreter            | You (the developer)                  |
| What it protects?  | Python internals (memory ops) | Your own shared data (like counters) |
| Can you remove it? | ❌ No (built into CPython)     | ✅ Yes (use only where needed)        |
| Visible to you?    | ❌ No                          | ✅ Yes                                |
| Purpose            | Interpreter safety            | Race condition prevention            |

---

## ✅ GIL and Asyncio

* `asyncio` uses **only one thread** — **no parallelism**, only **concurrency**.
* Since only one thread is used, GIL has **no negative impact**.
* Async programs switch tasks **cooperatively** using `await`, so there's no thread conflict.

---

## ✅ GIL and Threading

| Behavior            | Description                                                          |
| ------------------- | -------------------------------------------------------------------- |
| CPU-bound threading | ❌ GIL blocks true parallelism — threads run one at a time            |
| I/O-bound threading | ✅ GIL is released during I/O — good concurrency                      |
| No `join()`         | Main may exit before thread finishes (especially for daemon threads) |
| Daemon thread + GIL | Thread may be killed before using its chance to run                  |

---

## ✅ GIL and Multiprocessing

| Feature                                                | Description                                                     |
| ------------------------------------------------------ | --------------------------------------------------------------- |
| Each process has its own memory and Python interpreter | ✅ Each has its own GIL                                          |
| CPU-bound work                                         | ✅ True parallelism                                              |
| Memory shared?                                         | ❌ No — must use inter-process communication (Queue, Pipe, etc.) |
| Suitable for                                           | ML, data processing, heavy computation                          |

---

## ✅ Other Python Implementations (GIL behavior)

| Implementation | GIL?                         | Notes                                   |
| -------------- | ---------------------------- | --------------------------------------- |
| **CPython**    | ✅ Yes                        | Official, default interpreter           |
| **PyPy**       | ✅ Yes (but faster execution) | Optimized JIT compilation               |
| **Jython**     | ❌ No                         | Runs on Java VM, no GIL                 |
| **IronPython** | ❌ No                         | Runs on .NET CLR                        |
| **Stackless**  | ✅ Yes                        | Supports microthreads but still has GIL |

---

## ✅ Code-Based Evidence

1. **CPU-bound threads run slow** (because of GIL)
2. **I/O-bound threads run fast** (because GIL is released)
3. **Multiprocessing works best** for CPU-heavy tasks

---

## ✅ Final Verdict

| Goal                            | Use                    | GIL Affected? | Best Choice   |
| ------------------------------- | ---------------------- | ------------- | ------------- |
| High I/O tasks (API, sleep)     | `threading`, `asyncio` | No            | ✅ Works well  |
| CPU-heavy work (math, ML)       | `multiprocessing`      | No (bypassed) | ✅ Best option |
| True parallel threads (CPython) | ❌ Not possible         | Yes           | ❌ Blocked     |

---

# **✅ 8: `ThreadPoolExecutor` – Easy and Clean Multithreading**


Python’s `concurrent.futures.ThreadPoolExecutor` gives you a **high-level, clean API** to run threads **without manually starting and joining** each one.

It handles:

* Thread creation
* Task submission
* Result collection

---

### 🔧 Why Use `ThreadPoolExecutor`?

| Without ThreadPool          | With ThreadPoolExecutor    |
| --------------------------- | -------------------------- |
| Manual thread creation      | Automatic pool management  |
| Manual `start()` + `join()` | Just submit tasks          |
| No easy return values       | ✅ Can fetch results easily |

---

### 🧪 Example 1: Submitting multiple tasks

```python
from concurrent.futures import ThreadPoolExecutor
import time

def task(name):
    print(f"Started {name}")
    time.sleep(2)
    print(f"Finished {name}")
    return f"✅ {name} done"

with ThreadPoolExecutor(max_workers=3) as executor:
    futures = [executor.submit(task, f"Task-{i}") for i in range(5)]

    for future in futures:
        result = future.result()
        print("Result:", result)
```

---

### 🔍 What’s Happening:

* `max_workers=3`: At most 3 threads run at once.
* 5 tasks are submitted → pool schedules them.
* `future.result()` waits for each task to finish and gives the result.

---

### 🧠 Key Concepts:

| Term                        | Meaning                             |
| --------------------------- | ----------------------------------- |
| `ThreadPoolExecutor()`      | Creates a pool of reusable threads  |
| `executor.submit(fn, args)` | Starts a function in the background |
| `future.result()`           | Gets the result (waits if needed)   |

---


# **✅ 9: Threading vs Async vs Multiprocessing — When to Use What?**

Awesome — this is a crucial step in your journey.

---



Let’s now compare all three concurrency models in Python:

* `threading`
* `asyncio`
* `multiprocessing`

Each has **different strengths** depending on the type of task.

---

### 🧠 High-Level Summary

| Feature              | `threading`             | `asyncio`                        | `multiprocessing`                 |
| -------------------- | ----------------------- | -------------------------------- | --------------------------------- |
| **Concurrency type** | Real threads (OS-level) | Coroutine-based (single-thread)  | True parallelism (separate procs) |
| **GIL impact**       | ❌ Blocks CPU-bound      | ✅ Not affected (1 thread only)   | ✅ No GIL — full CPU usage         |
| **Good for**         | I/O tasks, GUIs         | I/O-heavy tasks (APIs, scraping) | CPU-heavy work (math, ML, etc.)   |
| **Parallelism**      | ❌ Limited (GIL)         | ❌ No (only concurrency)          | ✅ Yes                             |
| **Ease of use**      | Medium                  | Advanced but powerful            | Medium to High                    |
| **Overhead**         | Low                     | Very low                         | High (multiple processes)         |

---

### 🧪 Examples Based on Use-Case:

| Use Case                       | Best Choice              | Why?                        |
| ------------------------------ | ------------------------ | --------------------------- |
| Downloading 1000 files         | `asyncio` or `threading` | GIL is released during I/O  |
| Chatbot / Websocket handling   | `asyncio`                | Lightweight and scalable    |
| Data processing (large arrays) | `multiprocessing`        | CPU-intensive, GIL-free     |
| Web scraping                   | `asyncio`                | Sleep-based, async-friendly |
| GUI application                | `threading`              | GUI main loop + background  |
| Machine Learning training      | `multiprocessing`        | Full core usage needed      |

---

### 👶 Simple Analogy

| Model               | Analogy                                                           |
| ------------------- | ----------------------------------------------------------------- |
| **Threading**       | Multiple roommates in the same house sharing a kitchen (GIL)      |
| **Asyncio**         | One super-fast chef who works on many dishes without blocking     |
| **Multiprocessing** | Many chefs in separate houses — full independence and parallelism |

---

### ✅ TL;DR Recommendation Table

| Task Type | Best Tool                                       |
| --------- | ----------------------------------------------- |
| I/O-bound | `asyncio` or `threading`                        |
| CPU-bound | `multiprocessing`                               |
| Mixed     | Combine: e.g. `asyncio` + `ProcessPoolExecutor` |

---